In [1]:
filename = 'ibm01.hgr'
with open(f'./data/{filename}', 'r') as f:
    lines = f.readlines()
    num_nets, num_nodes = map(int, lines[0].split())
    hypergraph_vertices = list(range(num_nodes))
    hypergraph_edges = []
    for line in lines[1:]:
        hypergraph_edges.append([int(node) - 1 for node in line.split()])
print(num_nets, num_nodes)

14111 12752


In [2]:
from scipy.sparse import coo_matrix

row = []
col = []
value = []
for i, e in enumerate(hypergraph_edges):
    for v in e:
        row.append(v)
        col.append(i)
        value.append(1)
H = coo_matrix((value, (row, col)), shape=(num_nodes, num_nets), dtype=float)
print(H.shape)

(12752, 14111)


In [3]:
from scipy.sparse.linalg import svds
import numpy as np

U, S, Vt = svds(H, k=2, which='LM', random_state=42, solver='lobpcg', maxiter=10000)
U = U[:, np.argsort(S)[::-1]]
for i in range(U.shape[1]):    
    if U[np.argmax(np.absolute(U[:,i])),i] < 0:
        U[:,i] = -U[:,i]
print(U.shape, S.shape, Vt.shape)
print(S)

(12752, 2) (2,) (2, 14111)
[9.16374985 9.19233672]


In [4]:
import torch

# Create the hyperedge_index tensor
hyperedge_index = torch.tensor(np.array([
    np.concatenate(hypergraph_edges),
    np.repeat(np.arange(len(hypergraph_edges)), [len(e) for e in hypergraph_edges])
]), dtype=torch.long)

print(hyperedge_index.shape)

torch.Size([2, 50566])


In [5]:
from utils import create_clique_expansion_graph, compute_topological_features, create_partition_id_feature

adj_matrix, node_degree, pin_count = create_clique_expansion_graph(hypergraph_vertices, hypergraph_edges)
clique_topo_features = compute_topological_features(adj_matrix, 2, True, False)
star_topo_features = torch.tensor(U.copy(), dtype=torch.float)
partition_feature = create_partition_id_feature(len(hypergraph_vertices), filename)
features = np.column_stack([clique_topo_features, star_topo_features, node_degree, pin_count, partition_feature])
D = torch.tensor(pin_count, dtype=torch.float).unsqueeze(1)
del adj_matrix, node_degree, pin_count, clique_topo_features, star_topo_features, partition_feature
deg_feature_norm = np.linalg.norm(features[:, 4])
features[:, 0] = features[:, 0] / np.linalg.norm(features[:, 0]) * deg_feature_norm
features[:, 1] = features[:, 1] / np.linalg.norm(features[:, 1]) * deg_feature_norm
features[:, 2] = features[:, 2] / np.linalg.norm(features[:, 2]) * deg_feature_norm
features[:, 3] = features[:, 3] / np.linalg.norm(features[:, 3]) * deg_feature_norm
features[:, 5] = features[:, 5] / np.linalg.norm(features[:, 5]) * deg_feature_norm
features[:, 6] = features[:, 6] / np.linalg.norm(features[:, 6]) * deg_feature_norm
print(features.shape)

*******************************************************************************
 HMETIS 2.0pre1  Copyright 1998-2007, Regents of the University of Minnesota

HyperGraph Information -----------------------------------------------------
 Name: ./data/ibm01.hgr, #Vtxs: 12752, #Hedges: 14111, #Cons: 1

Options --------------------------------------------------------------------
 ptype=rb, ctype=H1, rtype=FAST, otype=cut, dbglvl=8
 kwayrefine: No, reconstruct: No, fixed: No
 Nruns: 1, NV-cycles: 1, CMaxNet: 50, RMaxNet: 50, Seed: 42
 #Parts: 2, UBfactor:   2.00

Partitioning... ------------------------------------------------------------

  Bisecting a hgraph of size [vertices=12752, hedges=14111, balance=0.50]

    The mincut for this bisection =     204.00 (average = 204.0) (balance = [0.49]
 --------------------------------------------------------------------------
  Summary for the 2-way partition:
                Hyperedge Cut:        204.00		(minimize)
      Sum of External Degrees:  

In [6]:
from models import NewHyperData

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
x = torch.tensor(features, dtype=torch.float)
data = NewHyperData(x=x, hyperedge_index=hyperedge_index)
data = data.to(device)
W = torch.sparse_coo_tensor(hyperedge_index, torch.ones(hyperedge_index.shape[1]), (num_nodes, num_nets)).to(device)
D = D.to(device)
print(device)
print(W.shape)
print(D.shape)
print(data.x.shape)

cuda
torch.Size([12752, 14111])
torch.Size([12752, 1])
torch.Size([12752, 7])


In [7]:
from models import GraphPartitionModel

input_dim = 7
latent_dim = 64
hidden_dim = 256
num_partitions = 2
num_epochs = 50
model = GraphPartitionModel(input_dim, hidden_dim, latent_dim, num_partitions, True)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
model.train()

for epoch in range(num_epochs):
    optimizer.zero_grad()
    Y = model(data)
    loss, kl_loss, hyperedge_cut_loss, balance_loss = model.combined_loss(Y, W, D)
    loss.backward()
    # torch.nn.utils.clip_grad_norm_(model.parameters(), 3)
    optimizer.step()
    print(f'Epoch {epoch + 1}, Loss: {loss.item()}, KL Loss: {kl_loss.item()}, Ncut Loss: {hyperedge_cut_loss.item()}, Balance Loss: {balance_loss.item()}')

Epoch 1, Loss: 348423.96875, KL Loss: 475.24908447265625, Ncut Loss: 0.7299512624740601, Balance Loss: 348420.09375
Epoch 2, Loss: 84215.6796875, KL Loss: 389.5168151855469, Ncut Loss: 0.7293936610221863, Balance Loss: 84211.8359375
Epoch 3, Loss: 10444.798828125, KL Loss: 360.5218505859375, Ncut Loss: 0.7292904853820801, Balance Loss: 10440.9716796875
Epoch 4, Loss: 8088.6748046875, KL Loss: 353.5520935058594, Ncut Loss: 0.7291035652160645, Balance Loss: 8084.8525390625
Epoch 5, Loss: 40892.8828125, KL Loss: 350.8944396972656, Ncut Loss: 0.729270339012146, Balance Loss: 40889.0625
Epoch 6, Loss: 64406.1171875, KL Loss: 351.213134765625, Ncut Loss: 0.7295522689819336, Balance Loss: 64402.29296875
Epoch 7, Loss: 77677.2421875, KL Loss: 355.051513671875, Ncut Loss: 0.729552149772644, Balance Loss: 77673.4140625
Epoch 8, Loss: 47744.625, KL Loss: 353.5704040527344, Ncut Loss: 0.7292856574058533, Balance Loss: 47740.80078125
Epoch 9, Loss: 30413.00390625, KL Loss: 359.64453125, Ncut Loss: 

In [8]:
model.eval()
print(model.hyperedge_cut_loss(Y, W).item())

9222.130859375


In [9]:
from utils import evalPoint

best_cut = float('inf')
best_imbalance = float('inf')
for tau in range(10):
    partitions = model.sample(data, m=1)
    cut, imbalance, partition_id = evalPoint(0, partitions[0], hypergraph_vertices, hypergraph_edges, 2, filename, True)
    if cut < best_cut:
        best_cut = cut
        best_imbalance = imbalance
        partition_id = partition_id / np.linalg.norm(partition_id) * np.linalg.norm(data.x[:, 4].cpu().numpy())
        data.x[:, 6] = torch.tensor(partition_id, dtype=torch.float).to(device)
    print(f'Tau: {tau}, Best Cut: {best_cut}, Imbalance: {best_imbalance:.3f}, Cut: {cut}, Imbalance: {imbalance:.3f}')
print(f'Best Cut: {best_cut}, Imbalance: {best_imbalance:.3f}')

Tau: 0, Best Cut: 307, Imbalance: 0.040, Cut: 307, Imbalance: 0.040
Tau: 1, Best Cut: 286, Imbalance: 0.033, Cut: 286, Imbalance: 0.033
Tau: 2, Best Cut: 202, Imbalance: 0.028, Cut: 202, Imbalance: 0.028
Tau: 3, Best Cut: 201, Imbalance: 0.039, Cut: 201, Imbalance: 0.039
Tau: 4, Best Cut: 201, Imbalance: 0.039, Cut: 252, Imbalance: 0.040
Tau: 5, Best Cut: 201, Imbalance: 0.039, Cut: 248, Imbalance: 0.028
Tau: 6, Best Cut: 201, Imbalance: 0.039, Cut: 242, Imbalance: 0.040
Tau: 7, Best Cut: 201, Imbalance: 0.039, Cut: 204, Imbalance: 0.040
Tau: 8, Best Cut: 201, Imbalance: 0.039, Cut: 211, Imbalance: 0.040
Tau: 9, Best Cut: 201, Imbalance: 0.039, Cut: 206, Imbalance: 0.040
Best Cut: 201, Imbalance: 0.039


In [12]:
from utils import evalPoint

def train_rl(model, data, hypergraph_vertices, hypergraph_edges, filename, 
             num_episodes=50, learning_rate=1e-4, max_imbalance=0.04):
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    best_cut = float('inf')
    best_imbalance = float('inf')
    best_partition = None
    
    # 记录训练历史
    history = {'episode': [], 'cut': [], 'imbalance': [], 'reward': [], 'advantage': []}
    
    # 记录前一次的cut
    previous_cut = None
    # 移动平均baseline
    reward_baseline = 0
    baseline_alpha = 0.9
    
    for episode in range(num_episodes):
        # 前向传播
        logits = model(data)
        action_probs = F.softmax(logits, dim=1)
        
        # 温度控制
        temperature = max(1.0 - episode / num_episodes, 0.5)
        action_probs = action_probs ** (1 / temperature)
        action_probs = action_probs / action_probs.sum(dim=1, keepdim=True)
        
        # 采样动作
        dist = torch.distributions.Categorical(action_probs)
        partition = dist.sample()
        log_probs = dist.log_prob(partition)
        
        # 使用传统算法优化分区
        partition_np = partition.cpu().detach().numpy()
        cut, imbalance, optimized_partition = evalPoint(
            episode, partition_np, hypergraph_vertices, hypergraph_edges, 2, filename, True)
        
        # 基本奖励
        base_reward = -cut
        imbalance_penalty = max(0, imbalance - max_imbalance) * 1000
        
        # 相对改进奖励
        if previous_cut is not None:
            # 计算相对改进
            if cut < previous_cut:
                improvement = previous_cut - cut
                # 非线性放大改进带来的收益
                improvement_bonus = 500 * (1 - math.exp(-improvement * 0.1))
                reward = base_reward + improvement_bonus - imbalance_penalty
            else:
                reward = base_reward - imbalance_penalty
        else:
            reward = base_reward - imbalance_penalty
        
        # 基线减法，减少方差
        advantage = reward - reward_baseline
        reward_baseline = baseline_alpha * reward_baseline + (1 - baseline_alpha) * reward
        
        # 发现新的最优解时给予额外奖励
        if cut < best_cut:
            best_cut = cut
            best_imbalance = imbalance
            best_partition = optimized_partition
            advantage += 100  # 额外奖励
        elif cut == best_cut and imbalance < best_imbalance:
            best_imbalance = imbalance
            best_partition = optimized_partition
            advantage += 50  # 稍小的额外奖励
        
        # 更新前一次的cut
        previous_cut = cut
        
        # 策略梯度更新
        loss = -log_probs.mean() * advantage
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # 记录历史
        history['episode'].append(episode)
        history['cut'].append(cut)
        history['imbalance'].append(imbalance)
        history['reward'].append(reward)
        history['advantage'].append(advantage)
        
        print(f'Episode {episode+1}/{num_episodes}, Cut: {cut}, Imbalance: {imbalance:.4f}, '
              f'Reward: {reward:.2f}, Advantage: {advantage:.2f}')
    
    return best_partition, best_cut, best_imbalance, history